In [1]:
from retraction_analysis.utils import get_retractions

r = get_retractions()

In [34]:
# retracted_pmids = {str(int(pmid)) for pmid in r["OriginalPaperPubMedID"].dropna()}.union(
#     {str(int(pmid)) for pmid in r["RetractionPubMedID"].dropna()}) - {0}
retracted_pmids = {str(int(pmid)) for pmid in r["OriginalPaperPubMedID"].dropna()} - {0}

In [49]:
len(retracted_pmids)

19039

In [35]:
import plotly.graph_objects as go
import pandas as pd
import psycopg

# CIViC

In [36]:
from civicpy import civic

In [37]:
evidence = civic.get_all_evidence()

In [38]:
civic_pmids = {i.source.citation_id for i in evidence}

In [39]:
retracted_pmids.intersection(civic_pmids)

{'25961751', '26287849'}

PMID 26287849 is associated with an erratum regarding disclosures:

> Vemurafenib in Multiple Nonmelanoma Cancers with BRAF V600 Mutations (Original Article, N Engl J Med 2015;373:726-736), Adjuvant Pertuzumab and Trastuzumab in Early HER2-Positive Breast Cancer (Original Article, N Engl J Med 2017;377:122-131). Disclosure information for Dr. Jose Baselga was inaccurate in two articles. In the 2017 article, the disclosure footnote (p. 131) should have included the following statement: “Dr. Baselga reports receiving personal and other fees from Northern Biologics, Infinity Pharmaceuticals, ApoGen Biotechnologies, PMV Pharma, Juno Therapeutics, Roche/Genentech, Novartis, Eli Lilly, Verastem, Chugai Pharmaceuticals, AstraZeneca, Sanofi-Aventis, Aragon Pharmaceuticals, Bayer Pharmaceuticals, and Seragon.” The article is correct at NEJM.org. With both the 2015 article and the 2017 article, updated disclosure forms for Dr. Baselga have been posted.

> Editor’s note: Dr. Baselga failed to disclose in these articles his multiple, substantial financial associations, which are now apparent in the updated disclosure forms. When we learned of this breach of trust, we conveyed our concern to Dr. Baselga’s institution, Memorial Sloan Kettering Cancer Center.

In [40]:
{i for i in evidence if i.source.citation_id in {'26287849'}}

{<CIViC evidence 1574>,
 <CIViC evidence 1575>,
 <CIViC evidence 1576>,
 <CIViC evidence 1598>,
 <CIViC evidence 3742>,
 <CIViC evidence 5905>,
 <CIViC evidence 6045>}

PMID 25961751 is retracted: https://pubmed.ncbi.nlm.nih.gov/36044518/

However, the only associated evidence item was rejected during curation.

In [41]:
retracted_civic_id = {i for i in evidence if i.source.citation_id in {'25961751'}}
retracted_civic_id

{<CIViC evidence 1582>}

In [42]:
list(retracted_civic_id)[0].status

'rejected'

✅

# DGIdb

Focusing on v5 claims initially

In [43]:
with psycopg.connect("dbname=dgidb user=postgres") as conn:
    with conn.cursor() as cur:
        result = cur.execute("SELECT * FROM publications")
        pubs = result.fetchall()

In [44]:
dgidb_pmids = {str(i[1]) for i in pubs}
len(dgidb_pmids)

12239

In [46]:
retracted_dgidb_pmids = retracted_pmids.intersection(dgidb_pmids)
len(retracted_dgidb_pmids)

24

In [48]:
len(retracted_dgidb_pmids) / len(dgidb_pmids)

0.0019609445216112425

❌

In [74]:
def lookup_interaction_claim(pmid: str):
    with psycopg.connect("dbname=dgidb user=postgres") as conn:
        with conn.cursor() as cur:
            result = cur.execute(f"""
            SELECT ic.id,
            s.source_db_name, 
            dc.name,
            gc.name,
            p.citation
    
            FROM interaction_claims ic
            LEFT JOIN sources s on ic.source_id = s.id
            JOIN interaction_claims_publications icp ON icp.interaction_claim_id = ic.id
            JOIN publications p ON icp.publication_id = p.id
            LEFT JOIN drug_claims dc ON ic.drug_claim_id = dc.id
            LEFT JOIN gene_claims gc ON ic.gene_claim_id = gc.id 
            WHERE p.pmid = {pmid};
            """)
            return result.fetchall()

_interaction_claim_details = []
for pmid in retracted_dgidb_pmids:
    _interaction_claim_details += lookup_interaction_claim(pmid)
interaction_claim_details = pd.DataFrame(interaction_claim_details, columns=["Claim ID", "Source", "Drug", "Gene", "Citation"]).drop_duplicates()
interaction_claim_details.to_csv("dgidb_retracted_publications.csv")
interaction_claim_details[["Drug", "Gene", "Citation"]]

,Drug,Gene,Citation
0,CELECOXIB,HSPA4,"Narayanan NK et al., 2006, Docosahexaenoic aci..."
1,DEXTRAN SULFATE,F11,"Yun TH et al., 2003, Thrombin activation of fa..."
2,PROPOFOL,APP,"Mandal PK et al., 2007, Molecular understandin..."
3,CHEMOPREVENTIVE,NFE2L2,"Jain AK et al., 2005, Nuclear import and expor..."
4,STI-571,XIAP,"Nimmanapalli R et al., 2001, Cotreatment with ..."
5,SAHA,BIRC5,"Kulp SK et al., 2006, Antitumor effects of a n..."
6,LY-294002,AKT1,"Won J et al., 2006, Small molecule-based rever..."
7,KU-55933,ATM,"Won J et al., 2006, Small molecule-based rever..."
8,APD-209,ADRENERGIC RECEPTOR BETA-2,"Mantovani G et al., 2013, Drugs in development..."
9,RESVERATROL,HSPB2,"Bezstarosti K et al., 2006, Differential prote..."


### Specifics (WIP)

**Kannan et al (Kinase Inhibitor, UVB -> NFE2L2)**

**Guo et al (Mapk Inhibitors -> AGER)**

Retracted: https://pubmed.ncbi.nlm.nih.gov/19594496/

**Iwase et al (5-FLUOROURACIL -> FAS)**

Retracted: https://pubmed.ncbi.nlm.nih.gov/36883770/

**Saura et al (Ipatasertib -> PTEN)**

> In the original version of this article (1), the stated disclosure of José Baselga is incorrect. The error has been corrected in the latest online HTML and PDF versions of the article. The authors regret this error.

https://aacrjournals.org/cancerdiscovery/article/8/11/1490/6211/Correction-A-First-in-Human-Phase-I-Study-of-the

**Guh et al (Ciglitazone -> PPARG, STAT3)**

Retracted: https://pubmed.ncbi.nlm.nih.gov/29772897/

**Luo et al (Leuprolide Acetate -> SMAD3)**

Retracted: https://pubmed.ncbi.nlm.nih.gov/25879939/

**Bezstarosti et al (Rseveratrol -> HSPB2)**

Retracted: https://pubmed.ncbi.nlm.nih.gov/23189335/

**Mantovani et al (APD-209 -> Adrenergic Receptor Beta-2)**

Retracted: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3839839/

**Won et al (LY-294002 -> AKT1, KU-55933 -> ATM)**

Retracted: https://www.nature.com/articles/nchembio0708-431

**Kulp et al (Saha -> BIRC5)**

Retracted: https://aacrjournals.org/clincancerres/article/25/9/2940/82483/Retraction-Antitumor-Effects-of-a-Novel

**Nimmanapalli et al (STI-571 -> XIAP)**

> The editors are publishing this note to alert readers to concerns about this article (1). Figure 5A and B use the same β-actin control panel. The β-actin control panel in Fig. 3A is similar to lanes 2 through 6 in the β-actin panels in Fig. 5A and B. The original data are no longer available for review, and the authors stand by the conclusions of the article.

https://aacrjournals.org/clincancerres/article/25/13/4195/81746/Editor-s-Note-Cotreatment-with-STI-571-Enhances

**Mandal et al (Propofol -> APP)**

Retracted: https://pubmed.ncbi.nlm.nih.gov/17935358/

**Narayanan et al (Celexocib -> HSPA4)**

Retracted: https://pubmed.ncbi.nlm.nih.gov/26853534/

**Yun TH et al (Dextran Sulfate -> F11)**

> Table 1 has been retracted by the authors for the following reasons. All of the authors with the exception of F. A. Baglia retract the specific data listed above because recent experiments conducted by Dipali Sinha, Sergei Shikov, Wenman Wu, and Syed Ahmad in the laboratory of Peter N. Walsh failrd to confirm the conclusion that activated platelets promote the activation of factor XI by thrombin. All of the other results reported in this paper are valid. A detailed explanation of the chronology of events leading to this retraction and the retraction of a paper from Biochemistry (Baglia, F. A., and Walsh, P. N. (1998) Prothrombin is a cofactor for the binding of factor XI to the platelet surface and for platelet-mediated factor XI activation by thrombin. Biochemistry 37 2271-2281) has been published in the journal Biochemistry (manuscript bi-2007-01501k, accepted July 27, 2007). We apologize to the readers, reviewers, and editors of the Journal of Biological Chemistry for publishing these erroneous data.

https://www.sciencedirect.com/science/article/pii/S0021925820586420?via%3Dihub